In [ ]:
import torch
from torch import nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
#定义超参数
EPOCH = 2               #所有数据训练的次数 EPOCH = 2 ，经过权衡训练时间和准确率后测试发现训练周期为2比较合适
BATCH_SIZE = 40         #批训练的手写数字图片数量
TIME_STEP = 28          #rnn的时间序列，具体表现在图像上就是一个个的像素点，因为每行有28个像素点所以时间序列是28
INPUT_SIZE = 28         #一行信息包括28个像素点
LR = 0.01               #学习率
DOWNLOAD_MNIST = True  #如果没有数据集就下载,如果有就将True改成False

In [ ]:
#MNIST数据集下载
#训练数据集
#如果没有数据集就下载数据集
train_data = dsets.MNIST(root='./mnist/',train=True, transform=transforms.ToTensor(), download=DOWNLOAD_MNIST) 
#测试数据集
test_data = dsets.MNIST(root='./mnist/',train=True, transform=transforms.ToTensor(),download=True)

In [ ]:
test_x = test_data.test_data.type(torch.FloatTensor)[:2000]/255.   #三个维度，手写图的长宽和样本个数
test_y = test_data.test_labels.numpy()[:2000]    #转换为numpy数组，选取2000个样本可以加快测试速度

In [ ]:
#数据加载器可在训练中进行小批量返回
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE)

In [ ]:
#定义网络模型
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        #因为用nn.LSTM比用nn.RNN()效果好，所以这里选择用nn.LSTM，
        # hidden_size是rnn隐藏单元，num_layers是rnn的层数，经测试选择两层比较好
        #输入和输出的批量大小为 1s 维度
        self.rnn = nn.LSTM(input_size=INPUT_SIZE,hidden_size=64,num_layers=2,batch_first=True)
        self.out = nn.Linear(64, 10)  #10个数字的分类

    def forward(self, x):
        #x有三个维度：批次、时间步长、输入大小
        #r_out有三个维度：批处理数据量、时间步长、输出大小
        #h_n：网络层数，批处理数据量，隐藏层结点个数)
        #h_c shape 网络层数，批处理数据量，隐藏层结点个数
        r_out, (h_n, h_c) = self.rnn(x, None)   #None 表示初始隐藏状态

        #在最后一个时间步选择r_out
        out = self.out(r_out[:, -1, :])
        return out

In [ ]:
rnn = RNN() #调用模型
print(rnn)  #查看模型结构

In [ ]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   #选择优化器
loss_func = nn.CrossEntropyLoss()                       #定义损失函数

In [ ]:
#训练，测试
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):  
        b_x = b_x.view(-1, 28, 28)
        #正向传播
        output = rnn(b_x)                               #一个banch data经过rnn网络训练后的输出
        loss = loss_func(output, b_y)                   #交叉熵损失函数
        #反向传播
        optimizer.zero_grad()                           #训练步骤的梯度
        loss.backward()                                 #反向传播，计算梯度
        optimizer.step()                                #设置学习率

        if step % 50 == 0:
            test_output = rnn(test_x)                   #样本，时间步长，输入大小
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y).astype(int).sum()) / float(test_y.size)
            print('Epoch: ', epoch, '| 训练误差: %.4f' % loss.data.numpy(), '| 准确率: %.2f' % accuracy)

In [ ]:
#显示20个待预测数据和真实数据
test_output = rnn(test_x[:20].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].data.numpy()
print(pred_y, '需要预测的数据')
print(test_y[:20], '真实数据')